# Application of Graphs on HGCAL data

## Augmenting the data for application into graphs

In [84]:
#Importing the required packages
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import uproot
import torch

In [85]:
# Reading the root file
file=uproot.open("ntuple_pi+_100GeV_100keve.root:AllLayers")
#file.keys()
#Extracting the data
hit_x=file['hit_x'].array(library='ak')
hit_y=file['hit_y'].array(library='ak')
hit_z=file['hit_z'].array(library='ak')
hit_l=file['hit_l'].array(library='ak')
hit_E=file['hit_E'].array(library='ak')

Here, we need to pass data for each event to create graphs for each event. This can be done very easily if we can store our data for each event into a dictionary.

In [86]:
events = []   # list of events, each event is a dict

num_events = len(hit_x)   # Number of events

for i in range(num_events):
    event = {
        "event_n":i,
        "hit_x": hit_x[i],
        "hit_y": hit_y[i],
        "hit_z": hit_z[i],
        "hit_E": hit_E[i],
        "hit_l": hit_l[i]
    }
    events.append(event)


In [87]:
#events

## A few tests

### Checking out the dictionary

In [88]:
#Checking for each event
events[0] # The data for the event-1

{'event_n': 0,
 'hit_x': <Array [0, 0, 0, 0, 0, ..., -3.88, 3.88, -3.88, 5.82] type='404 * float64'>,
 'hit_y': <Array [0, 0, 0, 0, 0, ..., -1.12, 3.36, -3.36, 1.12] type='404 * float64'>,
 'hit_z': <Array [13.9, 14.8, 16.8, 17.7, ..., 138, 138, 138, 145] type='404 * float64'>,
 'hit_E': <Array [9.53e-05, 8.14e-05, ..., 0.000768, 0.000217] type='404 * float64'>,
 'hit_l': <Array [1, 2, 3, 4, 5, 6, 6, ..., 37, 37, 38, 38, 38, 39] type='404 * int32'>}

In [89]:
events[0]['hit_x'] # The hit_x values for the first event

<Array [0, 0, 0, 0, 0, ..., -3.88, 3.88, -3.88, 5.82] type='404 * float64'>

### Testing the masking

In [90]:
#Testing how to implement the mask
hit_E_ev=events[0]["hit_E"]
hit_x_ev=events[0]["hit_x"]
mask=hit_E_ev>0
hit_x_ev[mask]

<Array [0, 0, 0, 0, 0, ..., -3.88, 3.88, -3.88, 5.82] type='404 * float64'>

### Testing the nearest neighbours

In [91]:
from sklearn.neighbors import NearestNeighbors
test_event=events[0]
test_x=test_event["hit_x"][mask]
test_y=test_event["hit_y"][mask]
test_z=test_event["hit_z"][mask]
X=np.transpose(np.vstack([test_x,test_y,test_z]))
#X=np.vstack([test_x,test_y,test_z])

In [92]:
nbrs_test = NearestNeighbors(n_neighbors=8, algorithm='ball_tree').fit(X)

distances_test, indices_test = nbrs_test.kneighbors(X)

indices_test

array([[  0,   1,   2, ...,   5,   6,   7],
       [  1,   0,   2, ...,   5,   6,   7],
       [  2,   3,   1, ...,   5,   7,   6],
       ...,
       [401, 403, 400, ..., 402, 386, 387],
       [402, 400, 387, ..., 397, 396, 394],
       [403, 401, 400, ..., 385, 386, 387]])

In [93]:
indices_test[:,0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [94]:
neighbors_test=indices_test[:,1:8+1]

In [95]:
len(X)

404

In [96]:
#for i in range(len(X)):
#    for j in neighbors_test[i]:
#        print(j)

In [97]:
#Build the edge list:
edges=[]
for i in range(len(X)):
    for j in neighbors_test[i]:
        edges.append([i,j])

In [98]:
edges_ar=np.array(edges)
edges_ar.shape

(2828, 2)

In [99]:
edge_indices=torch.tensor(edges,dtype=torch.long).t().contiguous()
#edge_indices=torch.tensor(edges,dtype=torch.long)

In [100]:
edge_indices

tensor([[  0,   0,   0,  ..., 403, 403, 403],
        [  1,   2,   3,  ..., 385, 386, 387]])

## Creating the event graphs

In [101]:
#Importing the required packages
from torch_geometric.data import Data
import torch

In [244]:
#Creating graphs for each event
from sklearn.neighbors import NearestNeighbors
def create_graph(event,k=9):
    # event: dictionary containing the data for 1 event
    #Extracting the data for the points where Energy value of hits is non zero
    mask=event["hit_E"]>0
    x=np.array(event["hit_x"])[mask]
    y=np.array(event["hit_y"])[mask]
    z=np.array(event["hit_z"])[mask]
    l=np.array(event["hit_l"])[mask]
    E=np.array(event["hit_E"])[mask]
    
    #Developing the node feature matrix
    node_features=torch.tensor(np.transpose(np.vstack([x,y,z,E])),dtype=torch.float)
    
    #3D coordinates for distance
    coords=np.vstack([x,y,z]).T
    N=len(coords)
    
    #Determining edges
    edges=[]
    
    for i in range(N):
        #Masking for allowed layers
        #allowed=np.where((l>=l[i]-1) & (l<=l[i]+1))[0]
        allowed=np.where((l>=l[i]-2) & (l<=l[i]+2))[0]
        allowed=allowed[allowed!=i] # removes itself
        allowed=allowed[E[allowed]>0] # removes zero energy neighbors
        if len(allowed)==0:
            continue # Skip for isolate hit
        #Applying KNN on allowed neighbors
        nbrs=NearestNeighbors(n_neighbors=min(k,len(allowed)),algorithm='auto').fit(coords[allowed])
        distances,indices=nbrs.kneighbors([coords[i]])
        neighbors=allowed[indices[0]]
        if i==76:
            print(distances,indices,neighbors)
            print(coords[allowed].shape)
        #Building undirected edges
        for j in neighbors:
            edges.append([i,j])
            edges.append([j,i])
    edge_index=torch.tensor(edges,dtype=torch.long)
    data=Data(x=node_features,edge_index=edge_index.t())
    return data

In [245]:
#Testing for our test event
test_event=events[0]
graph=create_graph(test_event,k=9)

[[1.006     1.12      1.12      1.12      1.12      1.12      1.12
  1.5054687 1.5054687]] [[11 36 37 31 34 32 35 19 17]] [56 82 83 77 80 78 81 64 62]
(96, 3)


In [226]:
graph

Data(x=[404, 4], edge_index=[2, 6382])

In [227]:
#Creating the graphs
#graph=[create_graph(event,k=8) for event in events]

In [228]:
events_test=[events[i] for i in range(10)]
#events_test

In [229]:
graph=[create_graph(event,k=8) for event in events_test]

[[1.006     1.12      1.12      1.12      1.12      1.12      1.12
  1.5054687]] [[11 37 34 31 36 32 35 19]] [56 83 80 77 82 78 81 64]
[[1.022      1.12       1.12       1.12       1.12       1.12
  1.12       1.51620711]] [[18 45 38 44 42 43 39 24]] [56 84 77 83 81 82 78 62]
[[1.12       1.9398969  1.9398969  1.9398969  2.13431488 2.13431488
  2.13431488 2.21847357]] [[34 31 36 43 18 12 20 51]] [75 72 78 85 59 53 61 93]
[[1.12      1.12      1.12      1.12      1.9398969 1.9398969 1.9398969
  1.9398969]] [[42 58 46 55 39 56 63 33]] [70 87 74 84 67 85 92 61]
[[0.89       1.12       1.12       1.12       1.12       1.12
  1.12       1.43055933]] [[87 54 59 47 62 50 49 91]] [113  80  85  72  88  75  74 117]
[[1.12      1.12      1.12      1.9398969 1.9398969 1.9398969 2.24
  2.24     ]] [[30 42 51 33 23 34 25 24]] [65 78 87 68 58 69 60 59]
[[1.12       1.12       1.12       1.9398969  1.9398969  2.24
  2.96324147 2.96324147]] [[24 25 23 19 22 18 14 21]] [75 77 74 70 73 69 65 72]
[[1.9398

In [233]:
graph[9]

Data(x=[739, 4], edge_index=[2, 11762])

### Validating the graph

In [208]:
data=create_graph(test_event,k=8)

In [209]:
ci=int(np.argmax(data.x[:,3].numpy()))

In [210]:
coords=data.x[:,:3].numpy()
coords

array([[  0.       ,   0.       ,  13.8625   ],
       [  0.       ,   0.       ,  14.7525   ],
       [  0.       ,   0.       ,  16.7675   ],
       ...,
       [  3.879794 ,   3.36     , 137.781    ],
       [ -3.879794 ,  -3.36     , 137.781    ],
       [  5.8196907,   1.12     , 145.1885   ]], dtype=float32)

In [211]:
p=coords[ci]

In [212]:
#dists=np.linalg.norm(coords-p,axis=1)
diff=coords-p
dists=np.sum(diff*diff,axis=1)

In [237]:
distance=np.linalg.norm(coords[57]-p)
distance

1.5054691

In [213]:
sorted_indices=np.argsort(dists)

In [214]:
#sorted_indices = sorted_indices[sorted_indices != ci]

In [215]:
nearest_nbrs=[idx for idx in sorted_indices[1:8+1]]
nearest_nbrs

[56, 78, 80, 81, 82, 77, 83, 57]

In [216]:
edge_index=data.edge_index.numpy()
#edge_index
src=edge_index[0]
dst=edge_index[1]
in_graph_edges=[j for j in nearest_nbrs if np.any(((src==ci) & (dst==j)) | ((src==j) & (dst==ci)))]
print(f"Neighbors found in graph= {in_graph_edges}")

Neighbors found in graph= [56, 78, 80, 81, 82, 77, 83]


In [218]:
edge_index = data.edge_index.numpy()
src = edge_index[0]
dst = edge_index[1]

# Create set of directed and undirected pairs
edge_set = set(zip(src, dst)) | set(zip(dst, src))

in_graph_edges = [j for j in nearest_nbrs if (ci, j) in edge_set]
print("Neighbors found in graph:", in_graph_edges)


Neighbors found in graph: [56, 78, 80, 81, 82, 77, 83]


In [220]:
edges = set(map(tuple, data.edge_index.numpy().T))
in_graph_edges = [j for j in nearest_nbrs if (ci, j) in edges or (j, ci) in edges]
in_graph_edges

[56, 78, 80, 81, 82, 77, 83]

In [217]:
#Verify the nearest indices from sklearn
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=9, algorithm='auto').fit(coords)
distances, indices = knn.kneighbors(coords)

print("SKLearn neighbors for node 76:", indices[76].tolist())


SKLearn neighbors for node 76: [76, 56, 77, 78, 82, 83, 81, 80, 57]
